In [ ]:
from tasks import bgm
import ramanchada2 as rc2
import matplotlib.pyplot as plt

In [ ]:
import os
root_folder = 'D:/nina/OneDrive/'
folder1 = os.path.join(root_folder,'CHARISMA - RUNTIME/WP 3 - TF3_RoundRobin',
                       'Review working documents','Peak fitting','b.Tasks',
                       'Task [Nico] Peak shape - resolution _ in preparation','Sharpening test',
                       '230411-2_NeLamp_514nm_Objx20')

input_files = ['230411-2_NeLamp_514nm_Objx20_Slit010um.txt',
        '230411-2_NeLamp_514nm_Objx20_Slit100um.txt',
        '230411-2_NeLamp_514nm_Objx20_Slit130um.txt',
        '230411-2_NeLamp_514nm_Objx20_Slit150um.txt']

paths = list(map(lambda a : os.path.join(folder1,a), input_files))

In [ ]:
from IPython.display import display, clear_output
from ipywidgets import widgets, interact, interactive, fixed, interact_manual
import numpy as np 

axis1= None
axis2 = None


#paths = paths_demo
pp_widget = widgets.Dropdown(
    options=paths,
    value=paths[0],
    description='Path:',
    disabled=False,
    )

p_widget = pp_widget
#
#p_widget = path_widget
#otherwise see the previous cell
nsamples_widget = widgets.FloatSlider(min=500, max=100000, step=100, value=10000)
iter_widget = widgets.FloatSlider(min=1, max=5000, step=500, value=2000)

crop_widget = widgets.SelectionRangeSlider(
    options=np.arange(100,2000,10),
    index=(59, 63),
    description='Crop range',
    disabled=False
)
ncomponents_widget = widgets.IntSlider(min=1, max=50, step=1, value=2)

peaks_widget = widgets.Button(
    description='Find peaks',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

peaks_progress = widgets.Text(
    value='',
    placeholder='',
    description='Peaks finding:',
    disabled=True
)

out = widgets.Output()
display(out)


def crop(spe,xcrop):
    if xcrop is None:
        return spe
    crop_index = np.where((spe.x >=xcrop[0]) &  (spe.x <=xcrop[1]))
    spe.x = spe.x[crop_index]
    spe.y = spe.y[crop_index]
    return spe

def loadncrop(path,xcrop,_plot=True,ax1=None,ax2=None):
    spe = rc2.spectrum.from_local_file(path)
    peaks_progress.value = ""
    if _plot:
        spe.plot(fmt='r',label=os.path.basename(path),ax=ax1)
    spe = crop(spe,xcrop)
    spe = spe - spe.moving_minimum(16)    
    if _plot:
        spe.plot(fmt='b',label="cropped, baseline removed",ax=ax2)  
    return spe 

def update_path(*args):
    for arg in args:
        do = False
        path = None
        xcrop = None
        if arg["owner"] == p_widget:
            path = arg["new"] 
            xcrop = crop_widget.value
        elif arg["owner"] == crop_widget:
            xcrop = arg["new"]
            path = p_widget.value
    if path is None or xcrop is None:
        pass
    else:
        pass
        #loadncrop(path,xcrop)
   
    

def find_peaks(val):
    peaks_widget.description = "in progress ..."
    spe = loadncrop(p_widget.value,crop_widget.value,_plot=False)
    peaks,model,n_peaks,hist_dist,index,xsamples = bgm.bgm_findpeaks(spe,xcrop=crop_widget.value,n_samples=nsamples_widget.value,
            n_components=ncomponents_widget.value,
            max_iter=iter_widget.value,sampling_coeff = None,plot=True,remove_baseline=False)
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,2))
    spe.plot(fmt='r',label=os.path.basename(p_widget.value),ax=ax1)
    ax1.twinx().hist(xsamples,bins=50,density=True)        
    #ax2.plot(xsamples,hist_dist.cdf(xsamples),'.')
    
    ax2.plot(spe.x[index],model.predict_proba(list(map(lambda e: [e], spe.x[index]))),'-')
    #ax2.plot(spe.x[index],hist_dist.cdf(spe.x[index]),'.')             
    peaks["domain"] = p_widget.value     
    display(peaks[0:n_peaks])
    peaks_widget.description = "Find peaks!"


#crop_widget.observe(update_path, 'value')
#p_widget.observe(update_path, 'value')
peaks_widget.observe(find_peaks, 'value')


def printer(path,xcrop,nsamples,niter,ncomponents):
    fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,2))       
    loadncrop(path,xcrop,_plot=True,ax1=ax1,ax2=ax2)
    
 
peaks_widget.on_click(find_peaks)
display(peaks_widget)

interact(printer,path=p_widget, xcrop=crop_widget, nsamples = nsamples_widget, niter = iter_widget, ncomponents = ncomponents_widget);    
